In [2]:
from pyspark.sql.functions import substring,conv,col,min,max,udf,lit
   
d1 = spark.read.csv('/user/maxnet/database/sig.db/term_desc_all',sep='\x01')
d1 = d1.withColumnRenamed('_c0','term_id')\
       .withColumnRenamed('_c1','brand')\
       .withColumnRenamed('_c2','cn_name')\
       .withColumnRenamed('_c3','en_name')\
       .withColumnRenamed('_c4','type1')\
       .withColumnRenamed('_c5','type2')\
       .withColumnRenamed('_c6','os')\
       .withColumnRenamed('_c7','dtime')\
       .withColumnRenamed('_c8','price')\
       .withColumnRenamed('_c9','remarks')\
       .withColumnRenamed('_c10','prio')

d2 = spark.read.csv('/user/maxnet/database/sig.db/term_oui',sep='\x01')
d2 = d2.withColumnRenamed('_c0','mac_6').withColumnRenamed('_c1','manu')

d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
d = d.withColumn('m1_6',substring(d.mac,1,6))

tmp_1 = d.join(d2,d.m1_6==d2.mac_6)
df = tmp_1.join(d1,tmp_1.id == d1.term_id).select('mac','id','manu','brand','prio').distinct().dropna()
df = df.filter(df.prio == 4).select('mac','id','brand').distinct().dropna()
df = df.filter(df.brand == '小米')

df = df.withColumn('m1',substring(df.mac,1,6)).withColumn('m2',substring(df.mac,7,6))
df = df.withColumn('p',conv(df.m2, 16, 10))
df = df.select('mac','id','m1','m2',col('p').cast('bigint'))

In [2]:
df.groupBy('m1').count().sort('count',ascending=False).show()

+------+-------+
|    m1|  count|
+------+-------+
|9487E0|1217740|
|9C2EA1| 782031|
|F4F5DB| 712422|
|A45046| 697953|
|4C49E3| 663933|
|80AD16| 634729|
|482CA0| 623304|
|D86375| 610537|
|38A4ED| 598649|
|B0E235| 589208|
|ACC1EE| 575838|
|742344| 567940|
|508F4C| 559196|
|00EC0A| 535711|
|ECD09F| 516090|
|7802F8| 514343|
|E446DA| 504217|
|50642B| 500685|
|2047DA| 489869|
|F460E2| 478567|
+------+-------+
only showing top 20 rows



In [2]:
d1 = df.filter(df.m1 == '9487E0')
d2 = df.filter(df.m1 == '9C2EA1')
d3 = df.filter(df.m1 == 'F4F5DB')
d4 = df.filter(df.m1 == 'A45046')
d5 = df.filter(df.m1 == '4C49E3')
d6 = df.filter(df.m1 == '80AD16')
d7 = df.filter(df.m1 == '482CA0')
d8 = df.filter(df.m1 == 'D86375')
d9 = df.filter(df.m1 == '38A4ED')
d10 = df.filter(df.m1 == 'B0E235')

In [3]:
#d1.sample(0.01).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d1',compression='gzip')
#d2.sample(0.1).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d2',compression='gzip')
#d3.sample(0.1).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d3',compression='gzip')
#d4.sample(0.1).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d4',compression='gzip')
#d5.sample(0.1).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d5',compression='gzip')
#d6.sample(0.1).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d6',compression='gzip')
#d7.sample(0.1).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d7',compression='gzip')
#d8.sample(0.1).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d8',compression='gzip')
#d9.sample(0.1).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d9',compression='gzip')
#d10.sample(0.1).write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/d10',compression='gzip')

In [3]:
def predict(x):
    
    from pyspark.sql.functions import substring,conv,col,min,max,udf,lit
   
    d3 = spark.read.parquet('/data/user/hive/warehouse/ian/tmp_9C2EA1')
    
    bottom = d3.select(min('p')).collect()[0]['min(p)']
    top = d3.select(max('p')).collect()[0]['max(p)'] 
    
    if x < bottom:
        return d3.filter(d3.p == bottom).select('id').collect()[0]['p']
    
    elif x > top:
        return d3.filter(d3.p == top).select('id').collect()[0]['p']
    
    else :
        tmp1 = d3.where(col('p').between((x-10),(x+10))).groupBy('id').count()
        tmp2 = d3.where(col('p').between((x-100),(x+100))).groupBy('id').count()
        tmp3 = d3.where(col('p').between((x-1000),(x+1000))).groupBy('id').count()
        tmp4 = d3.where(col('p').between((x-10000),(x+10000))).groupBy('id').count()
        tmp5 = d3.where(col('p').between((x-100000),(x+100000))).groupBy('id').count()
        return tmp1.union(tmp2).union(tmp3).union(tmp4).union(tmp5).groupBy('id').count().sort('count',ascending=False).collect()[0]['id'] 

In [4]:
import pandas as pd
from pyspark.sql.functions import substring,conv,col

#test = test.withColumn('m1',substring(test.mac,1,6)).withColumn('m2',substring(test.mac,7,6))
#test = test.withColumn('p',conv(test.m2, 16, 10))  
#test = test.select('mac','id','m1','m2',col('p').cast('bigint'))
test = spark.read.parquet('/data/user/hive/warehouse/ian/d2')

result = []
a = test.select('p').toPandas()['p'].values.tolist()
for x in a:
    result.append(predict(x))

mac = test.select('mac').toPandas()['mac'].values.tolist()
pre = spark.createDataFrame(pd.DataFrame({'mac':mac,'predict_id':result}))

pre = pre.withColumnRenamed('mac','mac1')
dd = test.join(pre,test.mac == pre.mac1,how='left')

In [5]:
from pyspark.sql.functions import when
dd = dd.withColumn('equal',when(dd.predict_id == dd.id, 1).otherwise(0))

In [6]:
dd.groupBy('equal').count().show()

+-----+-----+
|equal|count|
+-----+-----+
|    1|72616|
|    0| 5622|
+-----+-----+



In [7]:
72616/(5622+72616)

0.9281423349267619